# SHSAT Test Results Merge Notebook
[Return to project overview](final_project_overview.ipynb)

### Andrew Larimer, Deepak Nagaraj, Daniel Olmstead, Michael Winton (W207-4-Summer 2018 Final Project)

In this notebook, we will merge the data cleaned by the other "prep_" notebooks to create a single merged csv.

## Importing dataframes, indexed by our primary key
While school names may change or be input inconsistently, each school has a unique identifying DBN, sometimes referred to as a Location Code, to identify it. By importing each cleaned dataset with the DBN as the index, we are able to easily join them into a merged dataset.

In [1]:
import pandas as pd
import datetime
import re

# set default options
pd.set_option('display.max_columns', None)

In [2]:
# Load all datasets from CSV; when loading set index to the DBN column (to enforce uniqueness)
shsat_df = pd.read_csv('data_cleaned/cleaned_shsat_outcomes.csv', index_col="dbn")
print('SHSAT dataset:',shsat_df.shape) # confirm that it's (589, 5)

class_sizes_df = pd.read_csv('data_cleaned/cleaned_class_sizes.csv', index_col="dbn")
print('Class size dataset:', class_sizes_df.shape) # confirm that it's (494, 13)

explorer_df = pd.read_csv('data_cleaned/cleaned_explorer.csv', index_col="dbn")
print('Explorer dataset:', explorer_df.shape) # confirm that it's (596, 55)

selectiveness_df = pd.read_csv('data_cleaned/selectiveness.csv', index_col='dbn')
print('Selectiveness dataset:', selectiveness_df.shape) # confirm that it's (589, 2)

SHSAT dataset: (589, 5)
Class size dataset: (494, 13)
Explorer dataset: (596, 55)
Selectiveness dataset: (589, 2)


## Checking for duplicate entries.
We do a quick check to make sure there are no duplicate entries.

In [3]:
shsat_dups = shsat_df.index.duplicated()
class_sizes_dups = class_sizes_df.index.duplicated()
explorer_dups = explorer_df.index.duplicated()
selectiveness_dups = selectiveness_df.index.duplicated()

print("True or False: there are duplicated indices within any dataframes?")
print("{0}.".format(bool(sum(shsat_dups) + sum(class_sizes_dups) + sum(explorer_dups) + 
                         sum(selectiveness_dups))))

True or False: there are duplicated indices within any dataframes?
False.


## Inner joins for more complete data
We'll use inner joins to select the intersection of our datasets, thus only selecting for schools for which we have data from each dataframe.

In [4]:
merged_df = shsat_df.join(explorer_df, how="inner")
merged_df = merged_df.join(class_sizes_df, how="inner")
merged_df = merged_df.join(selectiveness_df, how="inner")
print("Merged Dataframe shape:",merged_df.shape)

Merged Dataframe shape: (464, 75)


In [5]:
merged_df.head()

,grade_7_enrollment,num_shsat_test_takers,offers_per_student,pct_test_takers,high_registrations,school_name,district,zip,community_school,economic_need_index,school_income_estimate,percent_ell,percent_asian,percent_black,percent_hispanic,percent_black__hispanic,percent_white,student_attendance_rate,percent_of_students_chronically_absent,rigorous_instruction_percent,rigorous_instruction_rating,collaborative_teachers_percent,collaborative_teachers_rating,supportive_environment_percent,supportive_environment_rating,effective_school_leadership_percent,effective_school_leadership_rating,strong_family_community_ties_percent,strong_family_community_ties_rating,trust_percent,trust_rating,student_achievement_rating,average_ela_proficiency,average_math_proficiency,grade_7_ela_all_students_tested,grade_7_ela_4s_all_students,grade_7_ela_4s_american_indian_or_alaska_native,grade_7_ela_4s_black_or_african_american,grade_7_ela_4s_hispanic_or_latino,grade_7_ela_4s_asian_or_pacific_islander,grade_7_ela_4s_white,grade_7_ela_4s_multiracial,grade_7_ela_4s_limited_english_proficient,grade_7_ela_4s_economically_disadvantaged,grade_7_math_all_students_tested,grade_7_math_4s_all_students,grade_7_math_4s_american_indian_or_alaska_native,grade_7_math_4s_black_or_african_american,grade_7_math_4s_hispanic_or_latino,grade_7_math_4s_asian_or_pacific_islander,grade_7_math_4s_white,grade_7_math_4s_multiracial,grade_7_math_4s_limited_english_proficient,grade_7_math_4s_economically_disadvantaged,sie_provided,in_bronx,in_brooklyn,in_manhattan,in_queens,in_staten,number_of_students_english,number_of_students_math,number_of_students_science,number_of_students_social_studies,number_of_classes_english,number_of_classes_math,number_of_classes_science,number_of_classes_social_studies,average_class_size_english,average_class_size_math,average_class_size_science,average_class_size_social_studies,school_pupil_teacher_ratio,gifted,selective
dbn,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20K187,266,251,75,91,1,THE CHRISTA MCAULIFFE SCHOOL\I.S. 187,20,11219,0,0.480,NaN,2,69,1,6,7,24,98.0,2.0,92.0,4.0,91.0,4.0,92.0,4.0,90.0,4.0,76.0,2.0,93.0,4.0,4.0,3.83,4.19,324,226,0,0,0,154,61,0,0,121,324,304,0,0,0,206,83,0,0,166,0,0,1,0,0,0,1052.0,934.0,1385.0,1313.0,42.0,37.0,49.0,46.0,25.047619,25.243243,28.265306,28.543478,17.6,0,1
21K239,422,336,46,78,1,MARK TWAIN I.S. 239 FOR THE GIFTED & TALENTED,21,11224,0,0.311,NaN,1,28,7,5,12,54,96.0,7.0,88.0,3.0,89.0,4.0,87.0,3.0,86.0,3.0,73.0,2.0,91.0,3.0,3.0,3.53,3.71,355,175,0,5,4,45,110,11,0,32,352,205,0,5,6,64,119,11,0,44,0,0,1,0,0,0,2710.0,2164.0,1169.0,1553.0,87.0,69.0,37.0,50.0,31.149425,31.362319,31.594595,31.060000,16.5,0,1
03M054,295,257,53,91,1,J.H.S. 054 BOOKER T. WASHINGTON,3,10025,0,0.221,NaN,2,9,8,12,20,64,96.0,5.0,85.0,NaN,83.0,NaN,87.0,NaN,80.0,2.0,84.0,3.0,90.0,3.0,4.0,3.46,3.70,250,135,0,4,9,16,101,0,0,14,251,170,0,5,6,23,129,0,0,17,0,0,0,1,0,0,868.0,849.0,849.0,849.0,32.0,30.0,30.0,30.0,27.125000,28.300000,28.300000,28.300000,17.2,0,1
15K051,379,280,33,75,1,M.S. 51 WILLIAM ALEXANDER,15,11215,0,0.248,NaN,2,12,10,19,29,57,97.0,3.0,87.0,4.0,88.0,3.0,89.0,4.0,85.0,3.0,74.0,2.0,90.0,3.0,4.0,3.50,3.49,354,186,0,8,12,27,134,0,0,13,354,188,0,5,10,28,140,0,0,15,0,0,1,0,0,0,1156.0,992.0,1125.0,1092.0,42.0,35.0,39.0,38.0,27.523810,28.342857,28.846154,28.736842,17.2,0,1
02M312,177,163,62,90,1,NEW YORK CITY LAB MIDDLE SCHOOL FOR COLLABORAT...,2,10011,0,0.308,NaN,1,47,2,7,10,34,98.0,2.0,88.0,3.0,91.0,4.0,90.0,4.0,83.0,3.0,85.0,3.0,92.0,3.0,4.0,3.62,3.94,173,85,0,0,0,45,27,8,0,27,173,130,0,0,0,74,41,11,0,43,0,0,0,1,0,0,NaN,535.0,541.0,NaN,NaN,19.0,18.0,NaN,NaN,28.157895,30.055556,NaN,16.0,0,1


In [6]:
print("Merged DF shape:",merged_df.shape)

Merged DF shape: (464, 75)


## Evaluating density
Let's take a look at how sparse our data is.

In [7]:
print("Total empty cells:",merged_df.isnull().sum().sum())
print("Percent null: {0:.3f}%".format(100*merged_df.isnull().sum().sum()/(merged_df.shape[0]*merged_df.shape[1])))

Total empty cells: 545
Percent null: 1.566%


Let's take a look at our worst offending rows and columns to see if anything stands out enough to be removed:

### Columns with Nulls

In [8]:
merged_df.isnull().sum()[merged_df.isnull().sum() > 0]\
    .sort_values(ascending=False)

school_income_estimate                    332
average_class_size_social_studies          19
number_of_classes_social_studies           19
number_of_students_social_studies          19
supportive_environment_rating              15
rigorous_instruction_rating                14
collaborative_teachers_rating              14
trust_rating                               12
effective_school_leadership_rating         12
strong_family_community_ties_rating        12
student_achievement_rating                 11
average_class_size_english                 10
number_of_students_english                 10
number_of_classes_english                  10
average_ela_proficiency                     3
student_attendance_rate                     3
percent_of_students_chronically_absent      3
rigorous_instruction_percent                3
collaborative_teachers_percent              3
supportive_environment_percent              3
effective_school_leadership_percent         3
strong_family_community_ties_perce

### Rows with Nulls

In [9]:
merged_df.isnull().sum(axis=1)[merged_df.isnull().sum(axis=1) > 0]\
    .sort_values(ascending=False)

dbn
15K839    22
03M291    19
28Q358    19
06M209    14
02M407    10
28Q332     8
02M177     8
02M933     8
18K763     8
31R028     8
01M839     7
02M255     7
30Q300     7
02M413     7
17K484     7
02M312     7
19K404     7
02M225     6
11X462     4
13K265     4
07X298     4
29Q327     4
08X376     4
01M332     4
08X562     4
03M860     4
19K678     4
03M054     4
29Q355     4
17K590     3
          ..
13K351     1
04M825     1
15K464     1
19K364     1
28Q167     1
28Q284     1
28Q287     1
29Q289     1
06M322     1
06M324     1
21K690     1
08X375     1
12X318     1
10X391     1
12X242     1
13K691     1
18K598     1
07X223     1
09X361     1
10X243     1
15K136     1
16K267     1
17K354     1
18K581     1
29Q192     1
01M378     1
08X448     1
10X308     1
12X190     1
20K187     1
Length: 339, dtype: int64

At the moment we don't see any of these as being offending enough to be removed, especially since we have already preserved some info from the 'school_income_estimate' feature.

## Save a dated file

To allow updates to the merged dataframe without disrupting work on models downstream until they are ready, we save a dated merged filename.

In [10]:
# Get the date to create the filename.
d = datetime.date
filename = "combined_data_{0}.csv".format( d.today().isoformat() )
print(filename)

combined_data_2018-08-07.csv


In [11]:
# check final shape (464,69)
merged_df.shape

(464, 75)

In [12]:
merged_df.to_csv("data_merged/{0}".format(filename))

## Save alternate dataset without class size information
Because we are missing class size data for approximately 100 schools, the `inner join` used to merge our dataframes drops those rows.  We will also save a variant of our dataset without the class size data, in case it turns out those features have low predictve value in our models.

In [13]:
no_class_size_df = shsat_df.join(explorer_df, how="inner")
no_class_size_df = no_class_size_df.join(selectiveness_df, how="inner")
print("Merged Dataframe shape (without class size data):",no_class_size_df.shape)

Merged Dataframe shape (without class size data): (556, 62)


### Verify that characteristics of the dataset (in terms of nulls) are similar to above

In [14]:
print("Total empty cells:",no_class_size_df.isnull().sum().sum())
print("Percent null: {0:.3f}%".format(100*no_class_size_df.isnull().sum().sum()/
                                      (no_class_size_df.shape[0]*no_class_size_df.shape[1])))

# check columns with nulls
no_class_size_df.isnull().sum()[no_class_size_df.isnull().sum() > 0]\
    .sort_values(ascending=False)

Total empty cells: 574
Percent null: 1.665%


school_income_estimate                    371
supportive_environment_rating              26
rigorous_instruction_rating                22
collaborative_teachers_rating              22
trust_rating                               21
strong_family_community_ties_rating        21
effective_school_leadership_rating         20
student_achievement_rating                 16
collaborative_teachers_percent              5
student_attendance_rate                     5
percent_of_students_chronically_absent      5
rigorous_instruction_percent                5
average_math_proficiency                    5
supportive_environment_percent              5
average_ela_proficiency                     5
effective_school_leadership_percent         5
strong_family_community_ties_percent        5
trust_percent                               5
economic_need_index                         5
dtype: int64

There characteristics are similar to our primary dataset, so we should feel comfortable using it if we do not need the class size data in our models.  Note that several of the columns with nulls in our primary merged dataset originally came from the class size data.  As a result, aside from `school_income_estimate`, our columns look quite good with respect to nulls.

In [15]:
# Get the date to create the filename.
filename = "combined_data_no_class_sizes_{0}.csv".format( d.today().isoformat() )
print(filename)

# check final shape (556, 62)
print(no_class_size_df.shape)

no_class_size_df.to_csv("data_merged/{0}".format(filename))

combined_data_no_class_sizes_2018-08-07.csv
(556, 62)
